# dart_fss
'01150' 등 특정 기업 정보 반환 에러

In [10]:
import pandas as pd
import cjw_dart

mariaDart = cjw_dart.MariaDart()


fs = mariaDart.getJamoo('005930 ', '20200530')
fs

import pandas as pd
import cjw_dart

mariaDart = cjw_dart.MariaDart()

tmp = mariaDart.getJamoo('005930', '20200101')
tmp


Quarterly reports:   0%|          | 0/2 [00:00<?, ?report/s]

{'bgn_de': '20200101',
 'corp_code': '00126380',
 'end_de': None,
 'financial statement': [{'title': '[D210000] Statement of financial position, '
                                   'current/non-current - Consolidated '
                                   'financial statements (Unit: KRW)'},
                         {'title': '[D310000] Income statement, by function of '
                                   'expense - Consolidated financial '
                                   'statements (Unit: KRW)'},
                         {'title': '[D410000] Statement of comprehensive '
                                   'income - Consolidated financial statements '
                                   '(Unit: KRW)'},
                         {'title': '[D520000] Statement of cash flows, '
                                   'indirect method - Consolidated financial '
                                   'statements (Unit: KRW)'}],
 'lang': 'ko',
 'report_tp': ['quarter'],
 'separate': False,
 'separator': True}

# FN guide 크롤링

In [6]:
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup as bs

def get_html_fnguide(ticker, gb):
    """    
    :param ticker: 종목코드 
    :param gb: 데이터 종류 (0 : 재무제표, 1 : 재무비율, 2: 투자지표)
    :return: 
    """
    url=[]

    url.append("https://comp.fnguide.com/SVO2/ASP/SVD_Finance.asp?pGB=1&gicode=A" + ticker + "&cID=&MenuYn=Y&ReportGB=&NewMenuID=103&stkGb=701")
    url.append("https://comp.fnguide.com/SVO2/ASP/SVD_FinanceRatio.asp?pGB=1&gicode=A" + ticker + "&cID=&MenuYn=Y&ReportGB=&NewMenuID=104&stkGb=701")
    url.append("https://comp.fnguide.com/SVO2/ASP/SVD_Invest.asp?pGB=1&gicode=A"+ ticker + "&cID=&MenuYn=Y&ReportGB=&NewMenuID=105&stkGb=701")

    if gb>2 :
        return None

    url = url[gb]
    try:

        req = Request(url,headers={'User-Agent': 'Mozilla/5.0'})
        html_text = urlopen(req).read()

    except AttributeError as e :
        return None

    return html_text

def ext_fin_fnguide_data(ticker,gb,item,n,freq="a"):
    """
    :param ticker: 종목코드
    :param gb: 데이터 종류 (0 : 재무제표, 1 : 재무비율, 2: 투자지표)
    :param item: html_text file에서 원하는 계정의 데이터를 가져온다.
    :param n: 최근 몇 개의 데이터를 가져 올것인지
    :param freq: Y : 연간재무, Q : 분기재무    
    :return: item의 과거 데이터
    """

    html_text = get_html_fnguide(ticker, gb)

    soup = bs(html_text, 'lxml')

    d = soup.find_all(text=item)

    if(len(d)==0) :
        return None

    #재무제표면 최근 3년을 가져오고 재무비율이면 최근 4년치를 가져온다.
    nlimit =3 if gb==0 else 4

    if n > nlimit :
        return None
    if freq == 'a':
        #연간 데이터
        d_ = d[0].find_all_next(class_="r",limit=nlimit)
        # 분기 데이터
    elif freq =='q':
        d_ = d[1].find_all_next(class_="r",limit=nlimit)
    else:
        d_ = None

    try :
        data = d_[(nlimit-n):nlimit]
        v = [v.text for v in data]

    except AttributeError as e:
        return None

    return(v)

profit_loss = pd.read_html(get_html_fnguide('005930',gb=0))[0] 
profit_loss

,IFRS(연결),2017/12,2018/12,2019/12,2020/12,전년동기,전년동기(%)
0,매출액,"2,395,754.0","2,437,714.0","2,304,009.0","2,368,070.0","2,304,009.0",2.8
1,매출원가,"1,292,907.0","1,323,944.0","1,472,395.0","1,444,883.0","1,472,395.0",-1.9
2,매출총이익,"1,102,847.0","1,113,770.0","831,613.0","923,187.0","831,613.0",11.0
3,판매비와관리비계산에 참여한 계정 펼치기,"566,397.0","524,903.0","553,928.0","563,248.0","553,928.0",1.7
4,영업이익,"536,450.0","588,867.0","277,685.0","359,939.0","277,685.0",29.6
5,영업이익(발표기준),"536,450.0","588,867.0","277,685.0","359,939.0","277,685.0",29.6
6,금융수익계산에 참여한 계정 펼치기,"97,374.0","99,993.0","101,616.0","122,676.0","101,616.0",20.7
7,금융원가계산에 참여한 계정 펼치기,"89,789.0","86,089.0","82,749.0","113,181.0","82,749.0",36.8
8,기타수익계산에 참여한 계정 펼치기,"30,107.0","14,850.0","17,787.0","13,841.0","17,787.0",-22.2
9,기타비용계산에 참여한 계정 펼치기,"14,196.0","11,420.0","14,147.0","24,889.0","14,147.0",75.9


In [66]:
import OpenDartReader

dart = OpenDartReader('601c51e82dfa122fa35a240ad5722593a0cbc2b4')
fs = dart.finstate_all(corp='005930', bsns_year=2019, reprt_code='11014')
fs_bs = fs[fs['sj_div']=='BS'] # 자본총계, 무형자산, 이연법인세자산, 이연법인세부채, 자본금, 현금및현금성자산, 단기차입금, 장기차입금
bs_list = ['자본총계', '무형자산', '이연법인세자산', '이연법인세부채', '자본금', '현금및현금성자산', '단기차입금', '장기차입금']
df = []
for b in bs_list:
    df.append(fs_bs[fs_bs['account_nm'] == b])
df = pd.concat(df)
df

,rcept_no,reprt_code,bsns_year,corp_code,sj_div,sj_nm,account_id,account_nm,account_detail,thstrm_nm,thstrm_amount,frmtrm_nm,frmtrm_amount,ord,thstrm_add_amount,frmtrm_q_nm,frmtrm_q_amount,frmtrm_add_amount
51,20191114001273,11014,2019,00126380,BS,재무상태표,ifrs-full_Equity,자본총계,-,제 51 기 3분기말,263442244000000,제 50 기말,247753177000000,54,NaN,NaN,NaN,NaN
17,20191114001273,11014,2019,00126380,BS,재무상태표,ifrs-full_IntangibleAssetsOtherThanGoodwill,무형자산,-,제 51 기 3분기말,15705674000000,제 50 기말,14891598000000,18,NaN,NaN,NaN,NaN
20,20191114001273,11014,2019,00126380,BS,재무상태표,ifrs-full_DeferredTaxAssets,이연법인세자산,-,제 51 기 3분기말,5461996000000,제 50 기말,5468002000000,21,NaN,NaN,NaN,NaN
39,20191114001273,11014,2019,00126380,BS,재무상태표,ifrs-full_DeferredTaxLiabilities,이연법인세부채,-,제 51 기 3분기말,17185000000000,제 50 기말,15162523000000,41,NaN,NaN,NaN,NaN
44,20191114001273,11014,2019,00126380,BS,재무상태표,ifrs-full_IssuedCapital,자본금,-,제 51 기 3분기말,897514000000,제 50 기말,897514000000,47,NaN,NaN,NaN,NaN
1,20191114001273,11014,2019,00126380,BS,재무상태표,ifrs-full_CashAndCashEquivalents,현금및현금성자산,-,제 51 기 3분기말,26604994000000,제 50 기말,30340505000000,2,NaN,NaN,NaN,NaN
25,20191114001273,11014,2019,00126380,BS,재무상태표,ifrs-full_ShorttermBorrowings,단기차입금,-,제 51 기 3분기말,12330248000000,제 50 기말,13586660000000,27,NaN,NaN,NaN,NaN
36,20191114001273,11014,2019,00126380,BS,재무상태표,dart_LongTermBorrowingsGross,장기차입금,-,제 51 기 3분기말,2002565000000,제 50 기말,85085000000,38,NaN,NaN,NaN,NaN


In [62]:
fs = dart.finstate(corp='011150', bsns_year=2019)
fs

,rcept_no,reprt_code,bsns_year,corp_code,stock_code,fs_div,fs_nm,sj_div,sj_nm,account_nm,thstrm_nm,thstrm_dt,thstrm_amount,frmtrm_nm,frmtrm_dt,frmtrm_amount,bfefrmtrm_nm,bfefrmtrm_dt,bfefrmtrm_amount,ord
0,20200330002361,11011,2019,00129013,011150,OFS,재무제표,BS,재무상태표,유동자산,제 44 기,2019.12.31 현재,"38,296,401,269",제 43 기,2018.12.31 현재,"41,144,155,050",제 42 기,2017.12.31 현재,"35,171,386,460",2
1,20200330002361,11011,2019,00129013,011150,OFS,재무제표,BS,재무상태표,비유동자산,제 44 기,2019.12.31 현재,"89,047,741,172",제 43 기,2018.12.31 현재,"85,371,101,922",제 42 기,2017.12.31 현재,"85,043,664,684",4
2,20200330002361,11011,2019,00129013,011150,OFS,재무제표,BS,재무상태표,자산총계,제 44 기,2019.12.31 현재,"127,344,142,441",제 43 기,2018.12.31 현재,"126,515,256,972",제 42 기,2017.12.31 현재,"120,215,051,144",6
3,20200330002361,11011,2019,00129013,011150,OFS,재무제표,BS,재무상태표,유동부채,제 44 기,2019.12.31 현재,"57,396,298,089",제 43 기,2018.12.31 현재,"58,921,962,680",제 42 기,2017.12.31 현재,"54,424,568,473",8
4,20200330002361,11011,2019,00129013,011150,OFS,재무제표,BS,재무상태표,비유동부채,제 44 기,2019.12.31 현재,"5,595,387,701",제 43 기,2018.12.31 현재,"5,826,325,867",제 42 기,2017.12.31 현재,"5,736,622,824",10
5,20200330002361,11011,2019,00129013,011150,OFS,재무제표,BS,재무상태표,부채총계,제 44 기,2019.12.31 현재,"62,991,685,790",제 43 기,2018.12.31 현재,"64,748,288,547",제 42 기,2017.12.31 현재,"60,161,191,297",12
6,20200330002361,11011,2019,00129013,011150,OFS,재무제표,BS,재무상태표,자본금,제 44 기,2019.12.31 현재,"18,065,386,500",제 43 기,2018.12.31 현재,"18,065,386,500",제 42 기,2017.12.31 현재,"18,065,386,500",15
7,20200330002361,11011,2019,00129013,011150,OFS,재무제표,BS,재무상태표,이익잉여금,제 44 기,2019.12.31 현재,"46,268,535,726",제 43 기,2018.12.31 현재,"44,616,934,586",제 42 기,2017.12.31 현재,"42,663,547,201",19
8,20200330002361,11011,2019,00129013,011150,OFS,재무제표,BS,재무상태표,자본총계,제 44 기,2019.12.31 현재,"64,352,456,651",제 43 기,2018.12.31 현재,"61,766,968,425",제 42 기,2017.12.31 현재,"60,053,859,847",22
9,20200330002361,11011,2019,00129013,011150,OFS,재무제표,IS,손익계산서,매출액,제 44 기,2019.01.01 ~ 2019.12.31,"149,168,950,259",제 43 기,2018.01.01 ~ 2018.12.31,"150,703,085,038",제 42 기,2017.01.01 ~ 2017.12.31,"164,629,714,604",24


In [55]:

dart.report(corp='005930', bsns_year=2019)

,rcept_no,reprt_code,bsns_year,corp_code,sj_div,sj_nm,account_id,account_nm,account_detail,thstrm_nm,thstrm_amount,frmtrm_nm,frmtrm_amount,bfefrmtrm_nm,bfefrmtrm_amount,ord,thstrm_add_amount
0,20200330003851,11011,2019,00126380,BS,재무상태표,ifrs-full_CurrentAssets,유동자산,-,제 51 기,181385260000000,제 50 기,174697424000000,제 49 기,146982464000000,1,NaN
1,20200330003851,11011,2019,00126380,BS,재무상태표,ifrs-full_CashAndCashEquivalents,현금및현금성자산,-,제 51 기,26885999000000,제 50 기,30340505000000,제 49 기,30545130000000,2,NaN
2,20200330003851,11011,2019,00126380,BS,재무상태표,dart_ShortTermDepositsNotClassifiedAsCashEquiv...,단기금융상품,-,제 51 기,76252052000000,제 50 기,65893797000000,제 49 기,49447696000000,3,NaN
3,20200330003851,11011,2019,00126380,BS,재무상태표,-표준계정코드 미사용-,단기매도가능금융자산,-,제 51 기,,제 50 기,,제 49 기,3191375000000,4,NaN
4,20200330003851,11011,2019,00126380,BS,재무상태표,-표준계정코드 미사용-,단기상각후원가금융자산,-,제 51 기,3914216000000,제 50 기,2703693000000,제 49 기,,5,NaN
5,20200330003851,11011,2019,00126380,BS,재무상태표,ifrs-full_CurrentFinancialAssetsAtFairValueThr...,단기당기손익-공정가치금융자산,-,제 51 기,1727436000000,제 50 기,2001948000000,제 49 기,,6,NaN
6,20200330003851,11011,2019,00126380,BS,재무상태표,dart_ShortTermTradeReceivable,매출채권,-,제 51 기,35131343000000,제 50 기,33867733000000,제 49 기,27695995000000,7,NaN
7,20200330003851,11011,2019,00126380,BS,재무상태표,-표준계정코드 미사용-,미수금,-,제 51 기,4179120000000,제 50 기,3080733000000,제 49 기,4108961000000,8,NaN
8,20200330003851,11011,2019,00126380,BS,재무상태표,-표준계정코드 미사용-,선급금,-,제 51 기,1426833000000,제 50 기,1361807000000,제 49 기,1753673000000,9,NaN
9,20200330003851,11011,2019,00126380,BS,재무상태표,-표준계정코드 미사용-,선급비용,-,제 51 기,2406220000000,제 50 기,4136167000000,제 49 기,3835219000000,10,NaN
